# Exercices : Protocoles d'Authentification

Ce notebook contient des exercices pratiques sur les protocoles d'authentification.

## 📚 Thèmes couverts

1. **Hachage de mots de passe** (SHA-256, salt, PBKDF2, bcrypt)
2. **Challenge-Response** (HMAC, prévention replay attacks)
3. **Multi-Factor Authentication** (TOTP)
4. **Analyse de vulnérabilités**

---

## Exercice 1 : Implémentation d'un système de hachage sécurisé

### Énoncé

Implémentez une classe `SecurePasswordManager` qui :
1. Hache les mots de passe avec **bcrypt**
2. Stocke les hashes dans un dictionnaire simulant une base de données
3. Vérifie les mots de passe lors de l'authentification
4. Permet de changer le coût de bcrypt

### Contraintes
- Utilisez `bcrypt` pour le hachage
- Le coût par défaut doit être **12**
- Ajoutez une méthode pour benchmarker différents coûts

In [ ]:
import bcrypt
import time

# TODO: Implémentez la classe SecurePasswordManager
class SecurePasswordManager:
    def __init__(self, cost=12):
        # TODO: Initialiser la base de données et le coût
        pass
    
    def register(self, username, password):
        # TODO: Hacher le mot de passe et le stocker
        pass
    
    def authenticate(self, username, password):
        # TODO: Vérifier le mot de passe
        pass
    
    def benchmark_cost(self, password, costs=[10, 12, 14]):
        # TODO: Tester différents coûts et mesurer le temps
        pass

### Solution

In [ ]:
# Solution
class SecurePasswordManager:
    def __init__(self, cost=12):
        self.cost = cost
        self.users_db = {}  # {username: hashed_password}
    
    def register(self, username, password):
        """Enregistre un utilisateur avec mot de passe haché"""
        if username in self.users_db:
            print(f"❌ Utilisateur '{username}' existe déjà")
            return False
        
        # Hache le mot de passe
        pwd_bytes = password.encode('utf-8')
        salt = bcrypt.gensalt(rounds=self.cost)
        hashed = bcrypt.hashpw(pwd_bytes, salt)
        
        self.users_db[username] = hashed
        print(f"✅ Utilisateur '{username}' enregistré (cost={self.cost})")
        return True
    
    def authenticate(self, username, password):
        """Vérifie le mot de passe d'un utilisateur"""
        if username not in self.users_db:
            print(f"❌ Utilisateur '{username}' inconnu")
            return False
        
        pwd_bytes = password.encode('utf-8')
        hashed = self.users_db[username]
        
        if bcrypt.checkpw(pwd_bytes, hashed):
            print(f"✅ Authentification réussie pour '{username}'")
            return True
        else:
            print(f"❌ Mot de passe incorrect pour '{username}'")
            return False
    
    def benchmark_cost(self, password, costs=[10, 12, 14]):
        """Benchmark différents coûts bcrypt"""
        print("\n=== Benchmark bcrypt cost ===")
        pwd_bytes = password.encode('utf-8')
        
        for cost in costs:
            start = time.time()
            salt = bcrypt.gensalt(rounds=cost)
            bcrypt.hashpw(pwd_bytes, salt)
            duration = time.time() - start
            
            print(f"Cost {cost:2d}: {duration:.3f}s ({duration*1000:.0f}ms)")

# Test de la solution
manager = SecurePasswordManager(cost=12)
manager.register("alice", "SuperSecretPassword123!")
manager.authenticate("alice", "SuperSecretPassword123!")  # Devrait réussir
manager.authenticate("alice", "WrongPassword")  # Devrait échouer

# Benchmark
manager.benchmark_cost("TestPassword123", costs=[10, 12, 14])

---

## Exercice 2 : Implémentation de TOTP (Time-based OTP)

### Énoncé

Implémentez une version simplifiée de TOTP qui :
1. Génère un code à 6 chiffres basé sur le timestamp actuel
2. Utilise HMAC-SHA1 (comme dans RFC 6238)
3. Accepte les codes dans une fenêtre de tolérance (±1 time step)

### Spécifications
- Time step : **30 secondes**
- Algorithme : **HMAC-SHA1**
- Nombre de chiffres : **6**
- Fenêtre de tolérance : **±1 step** (90 secondes au total)

In [ ]:
import hmac
import hashlib
import time
import struct
import base64

# TODO: Implémentez la fonction generate_totp
def generate_totp(secret, time_step=30, digits=6, timestamp=None):
    """
    Génère un code TOTP.
    
    Args:
        secret: Clé secrète (bytes ou string base32)
        time_step: Durée d'un time step en secondes
        digits: Nombre de chiffres du code
        timestamp: Timestamp à utiliser (None = maintenant)
    
    Returns:
        Code TOTP (string de 6 chiffres)
    """
    # TODO: Implémenter TOTP selon RFC 6238
    pass

# TODO: Implémentez la fonction verify_totp
def verify_totp(secret, code, tolerance=1, time_step=30):
    """
    Vérifie un code TOTP avec fenêtre de tolérance.
    
    Args:
        secret: Clé secrète
        code: Code à vérifier
        tolerance: Nombre de time steps avant/après à accepter
        time_step: Durée d'un time step
    
    Returns:
        True si le code est valide, False sinon
    """
    # TODO: Vérifier le code avec tolérance
    pass

### Solution

In [ ]:
# Solution
def generate_totp(secret, time_step=30, digits=6, timestamp=None):
    """Génère un code TOTP selon RFC 6238"""
    # Convertit secret si nécessaire
    if isinstance(secret, str):
        secret = base64.b32decode(secret.upper())
    
    # Calcule le counter (nombre de time steps depuis epoch)
    if timestamp is None:
        timestamp = time.time()
    counter = int(timestamp / time_step)
    
    # Convertit counter en bytes (8 bytes, big-endian)
    counter_bytes = struct.pack('>Q', counter)
    
    # Calcule HMAC-SHA1
    hmac_hash = hmac.new(secret, counter_bytes, hashlib.sha1).digest()
    
    # Dynamic Truncation (RFC 4226)
    offset = hmac_hash[-1] & 0x0F
    truncated = hmac_hash[offset:offset+4]
    code = struct.unpack('>I', truncated)[0] & 0x7FFFFFFF
    
    # Génère le code à N chiffres
    code = code % (10 ** digits)
    
    return f"{code:0{digits}d}"

def verify_totp(secret, code, tolerance=1, time_step=30):
    """Vérifie un code TOTP avec fenêtre de tolérance"""
    current_time = time.time()
    
    # Teste le code pour la fenêtre de temps actuelle ± tolerance
    for offset in range(-tolerance, tolerance + 1):
        timestamp = current_time + (offset * time_step)
        expected_code = generate_totp(secret, time_step, timestamp=timestamp)
        
        if hmac.compare_digest(code, expected_code):
            if offset == 0:
                print(f"✅ Code valide (fenêtre actuelle)")
            else:
                direction = "précédente" if offset < 0 else "suivante"
                print(f"✅ Code valide (fenêtre {direction}, offset={offset})")
            return True
    
    print(f"❌ Code invalide")
    return False

# Test de la solution
print("=" * 60)
print("TEST TOTP")
print("=" * 60)

# Génère une clé secrète
secret = base64.b32encode(b"test_secret_key_12345").decode('utf-8')
print(f"Secret (base32): {secret}")

# Génère un code
code = generate_totp(secret)
print(f"\nCode TOTP actuel: {code}")

# Vérifie le code immédiatement
print("\nVérification immédiate:")
verify_totp(secret, code)

# Vérifie un code incorrect
print("\nVérification avec mauvais code:")
verify_totp(secret, "000000")

# Teste la tolérance
print("\nTest de la tolérance (code de la période précédente):")
old_code = generate_totp(secret, timestamp=time.time() - 30)
print(f"Code de la période précédente: {old_code}")
verify_totp(secret, old_code, tolerance=1)

---

## Exercice 3 : Analyse de vulnérabilités

### Énoncé

Analysez le code suivant et identifiez **3 vulnérabilités** de sécurité.

```python
import hashlib

class VulnerableAuthSystem:
    def __init__(self):
        self.users = {}  # {username: password_hash}
    
    def register(self, username, password):
        # Hash le mot de passe avec SHA-256
        password_hash = hashlib.sha256(password.encode()).hexdigest()
        self.users[username] = password_hash
        return True
    
    def authenticate(self, username, password):
        if username not in self.users:
            return False
        
        # Vérifie le mot de passe
        password_hash = hashlib.sha256(password.encode()).hexdigest()
        
        if password_hash == self.users[username]:
            return True
        return False
```

### Questions

1. Quelles sont les **3 vulnérabilités** principales ?
2. Comment les corriger ?
3. Proposez une version sécurisée du code.

### Solution

#### Vulnérabilités identifiées

**1. Pas de salt** ❌
- Le hash SHA-256 est calculé directement sur le mot de passe
- Vulnérable aux **rainbow tables** (tables précalculées de hashes)
- Deux utilisateurs avec le même mot de passe auront le même hash

**2. Algorithme de hachage inadapté** ❌
- SHA-256 est trop rapide pour le hachage de mots de passe
- Un attaquant peut tester des **milliards de mots de passe par seconde**
- Devrait utiliser **bcrypt**, **scrypt** ou **Argon2**

**3. Comparaison non sécurisée** ❌
- Utilise `==` au lieu de `hmac.compare_digest()`
- Vulnérable aux **timing attacks** (l'attaquant mesure le temps de réponse)
- La comparaison s'arrête au premier caractère différent

#### Version sécurisée

In [ ]:
import bcrypt
import hmac

class SecureAuthSystem:
    def __init__(self, cost=12):
        self.users = {}  # {username: password_hash}
        self.cost = cost
    
    def register(self, username, password):
        """Enregistre un utilisateur avec bcrypt (salt automatique)"""
        # bcrypt génère automatiquement un salt unique
        password_bytes = password.encode('utf-8')
        salt = bcrypt.gensalt(rounds=self.cost)
        password_hash = bcrypt.hashpw(password_bytes, salt)
        
        self.users[username] = password_hash
        print(f"✅ Utilisateur '{username}' enregistré (bcrypt cost={self.cost})")
        return True
    
    def authenticate(self, username, password):
        """Authentifie un utilisateur de manière sécurisée"""
        if username not in self.users:
            print(f"❌ Utilisateur '{username}' inconnu")
            return False
        
        password_bytes = password.encode('utf-8')
        stored_hash = self.users[username]
        
        # bcrypt.checkpw utilise une comparaison timing-safe
        if bcrypt.checkpw(password_bytes, stored_hash):
            print(f"✅ Authentification réussie pour '{username}'")
            return True
        else:
            print(f"❌ Mot de passe incorrect pour '{username}'")
            return False

# Test du système sécurisé
print("=" * 60)
print("SYSTÈME SÉCURISÉ")
print("=" * 60)

secure_system = SecureAuthSystem(cost=12)
secure_system.register("alice", "password123")
secure_system.register("bob", "password123")  # Même mot de passe

# Vérifie que les hashes sont différents (grâce au salt)
print(f"\nHash Alice: {secure_system.users['alice'].hex()[:64]}...")
print(f"Hash Bob:   {secure_system.users['bob'].hex()[:64]}...")
print("\n👉 Les hashes sont différents malgré le même mot de passe (salt unique)")

# Test d'authentification
print("\n--- Tests d'authentification ---")
secure_system.authenticate("alice", "password123")  # Succès
secure_system.authenticate("alice", "wrongpassword")  # Échec

---

## Exercice 4 : Simulation d'attaque par dictionnaire

### Énoncé

Comparez la résistance de différents algorithmes de hachage face à une **attaque par dictionnaire**.

Testez :
1. SHA-256 sans salt
2. SHA-256 avec salt
3. PBKDF2 (10 000 itérations)
4. bcrypt (cost=12)

Utilisez un dictionnaire de 1000 mots de passe courants.

In [ ]:
import hashlib
import bcrypt
import time

# Dictionnaire de mots de passe courants (simulé)
common_passwords = [
    "password", "123456", "12345678", "qwerty", "abc123",
    "monkey", "letmein", "trustno1", "dragon", "baseball",
    "iloveyou", "master", "sunshine", "ashley", "bailey",
    "shadow", "superman", "password1", "princess", "admin"
] * 50  # Répète pour avoir 1000 mots

target_password = "sunshine"

print("=" * 60)
print("SIMULATION: Attaque par dictionnaire")
print("=" * 60)
print(f"Dictionnaire: {len(common_passwords)} mots de passe")
print(f"Mot de passe cible: '{target_password}'\n")

# 1. SHA-256 sans salt (TRÈS VULNÉRABLE)
print("--- Test 1: SHA-256 sans salt ---")
target_hash_sha256 = hashlib.sha256(target_password.encode()).hexdigest()

start = time.time()
for i, pwd in enumerate(common_passwords):
    pwd_hash = hashlib.sha256(pwd.encode()).hexdigest()
    if pwd_hash == target_hash_sha256:
        duration = time.time() - start
        print(f"✅ Trouvé en {i+1} tentatives ({duration*1000:.2f}ms)")
        print(f"   Vitesse: {(i+1)/duration:.0f} hashes/seconde")
        break

# 2. SHA-256 avec salt (Attaque ralentie mais possible)
print("\n--- Test 2: SHA-256 avec salt ---")
salt = b"random_salt_12345"
target_hash_salted = hashlib.sha256(salt + target_password.encode()).hexdigest()

start = time.time()
for i, pwd in enumerate(common_passwords):
    pwd_hash = hashlib.sha256(salt + pwd.encode()).hexdigest()
    if pwd_hash == target_hash_salted:
        duration = time.time() - start
        print(f"✅ Trouvé en {i+1} tentatives ({duration*1000:.2f}ms)")
        print(f"   Vitesse: {(i+1)/duration:.0f} hashes/seconde")
        break

# 3. PBKDF2 (Plus lent)
print("\n--- Test 3: PBKDF2 (10,000 iterations) ---")
iterations = 10000
target_hash_pbkdf2 = hashlib.pbkdf2_hmac(
    'sha256', target_password.encode(), salt, iterations
)

start = time.time()
for i, pwd in enumerate(common_passwords):
    pwd_hash = hashlib.pbkdf2_hmac('sha256', pwd.encode(), salt, iterations)
    if pwd_hash == target_hash_pbkdf2:
        duration = time.time() - start
        print(f"✅ Trouvé en {i+1} tentatives ({duration:.2f}s)")
        print(f"   Vitesse: {(i+1)/duration:.0f} hashes/seconde")
        break

# 4. bcrypt (BEAUCOUP plus lent)
print("\n--- Test 4: bcrypt (cost=12) ---")
print("⚠️  Ceci prendra du temps (c'est le but!)")
target_hash_bcrypt = bcrypt.hashpw(target_password.encode(), bcrypt.gensalt(rounds=12))

start = time.time()
tested = 0
for i, pwd in enumerate(common_passwords[:100]):  # Limite à 100 pour la démo
    if bcrypt.checkpw(pwd.encode(), target_hash_bcrypt):
        duration = time.time() - start
        print(f"✅ Trouvé en {i+1} tentatives ({duration:.2f}s)")
        print(f"   Vitesse: {(i+1)/duration:.2f} hashes/seconde")
        break
    tested = i + 1
else:
    duration = time.time() - start
    print(f"⏱️  Testé {tested} mots en {duration:.2f}s")
    print(f"   Vitesse: {tested/duration:.2f} hashes/seconde")

print("\n" + "=" * 60)
print("CONCLUSION")
print("=" * 60)
print("SHA-256 sans salt: TRÈS RAPIDE → Vulnérable (rainbow tables)")
print("SHA-256 avec salt: RAPIDE → Toujours vulnérable (brute force)")
print("PBKDF2: LENT → Ralentit les attaques")
print("bcrypt: TRÈS LENT → Excellent pour les mots de passe")
print("\n👉 Utilisez toujours bcrypt, scrypt ou Argon2 pour les mots de passe!")

---

## Exercice 5 : Implémentation d'un système 2FA complet

### Énoncé

Créez un système d'authentification complet avec :
1. **Authentification par mot de passe** (bcrypt)
2. **2FA avec TOTP** (optionnel par utilisateur)
3. **Codes de récupération** (backup codes)

### Fonctionnalités requises
- Enregistrement d'utilisateur avec mot de passe
- Activation du 2FA (génère secret TOTP + 10 codes de récupération)
- Authentification en 2 étapes (password + TOTP)
- Utilisation des codes de récupération (usage unique)

In [ ]:
import bcrypt
import secrets
import base64

class TwoFactorAuthSystem:
    def __init__(self):
        # Structure: {username: {
        #   'password_hash': bytes,
        #   '2fa_enabled': bool,
        #   'totp_secret': str,
        #   'backup_codes': set[str]
        # }}
        self.users = {}
    
    def register(self, username, password):
        """Enregistre un nouvel utilisateur"""
        if username in self.users:
            print(f"❌ Utilisateur '{username}' existe déjà")
            return False
        
        password_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt(rounds=12))
        
        self.users[username] = {
            'password_hash': password_hash,
            '2fa_enabled': False,
            'totp_secret': None,
            'backup_codes': set()
        }
        
        print(f"✅ Utilisateur '{username}' enregistré")
        return True
    
    def enable_2fa(self, username):
        """Active le 2FA pour un utilisateur"""
        if username not in self.users:
            print(f"❌ Utilisateur '{username}' inconnu")
            return None
        
        # Génère un secret TOTP
        secret_bytes = secrets.token_bytes(20)
        totp_secret = base64.b32encode(secret_bytes).decode('utf-8')
        
        # Génère 10 codes de récupération
        backup_codes = set()
        for _ in range(10):
            # Format: XXXX-XXXX-XXXX
            code = f"{secrets.randbelow(10000):04d}-{secrets.randbelow(10000):04d}-{secrets.randbelow(10000):04d}"
            backup_codes.add(code)
        
        self.users[username]['2fa_enabled'] = True
        self.users[username]['totp_secret'] = totp_secret
        self.users[username]['backup_codes'] = backup_codes
        
        print(f"✅ 2FA activé pour '{username}'")
        print(f"\nSecret TOTP (à scanner dans Google Authenticator):")
        print(f"   {totp_secret}")
        print(f"\nCodes de récupération (à conserver en lieu sûr):")
        for i, code in enumerate(sorted(backup_codes), 1):
            print(f"   {i:2d}. {code}")
        
        return totp_secret, backup_codes
    
    def authenticate_step1(self, username, password):
        """Première étape: vérification du mot de passe"""
        if username not in self.users:
            print(f"❌ Utilisateur '{username}' inconnu")
            return False
        
        user = self.users[username]
        
        if not bcrypt.checkpw(password.encode(), user['password_hash']):
            print(f"❌ Mot de passe incorrect")
            return False
        
        print(f"✅ Mot de passe correct")
        
        if user['2fa_enabled']:
            print(f"⚠️  2FA activé - Code TOTP requis")
            return "2fa_required"
        else:
            print(f"✅ Authentification réussie (pas de 2FA)")
            return True
    
    def authenticate_step2(self, username, code):
        """Deuxième étape: vérification TOTP ou code de récupération"""
        if username not in self.users:
            return False
        
        user = self.users[username]
        
        if not user['2fa_enabled']:
            print(f"❌ 2FA non activé pour cet utilisateur")
            return False
        
        # Vérifie si c'est un code de récupération
        if code in user['backup_codes']:
            user['backup_codes'].remove(code)  # Usage unique
            remaining = len(user['backup_codes'])
            print(f"✅ Code de récupération valide (reste {remaining} codes)")
            return True
        
        # Sinon, vérifie le code TOTP
        if verify_totp(user['totp_secret'], code, tolerance=1):
            return True
        else:
            print(f"❌ Code 2FA invalide")
            return False

# Démonstration complète
print("=" * 60)
print("SYSTÈME 2FA COMPLET")
print("=" * 60)

system = TwoFactorAuthSystem()

# 1. Enregistrement
print("\n--- Enregistrement ---")
system.register("charlie", "SuperSecure123!")

# 2. Activation du 2FA
print("\n--- Activation du 2FA ---")
totp_secret, backup_codes = system.enable_2fa("charlie")

# 3. Authentification sans 2FA (pour comparaison)
print("\n" + "=" * 60)
print("--- Authentification utilisateur sans 2FA ---")
system.register("dave", "AnotherPassword456!")
system.authenticate_step1("dave", "AnotherPassword456!")

# 4. Authentification avec 2FA
print("\n" + "=" * 60)
print("--- Authentification utilisateur avec 2FA ---")
result = system.authenticate_step1("charlie", "SuperSecure123!")

if result == "2fa_required":
    print("\n--- Étape 2: Vérification TOTP ---")
    current_totp = generate_totp(totp_secret)
    print(f"Code TOTP actuel: {current_totp}")
    system.authenticate_step2("charlie", current_totp)

# 5. Utilisation d'un code de récupération
print("\n" + "=" * 60)
print("--- Utilisation d'un code de récupération ---")
backup_code = list(backup_codes)[0]
print(f"Utilisation du code: {backup_code}")
system.authenticate_step1("charlie", "SuperSecure123!")
system.authenticate_step2("charlie", backup_code)

# Tentative de réutilisation (devrait échouer)
print("\n--- Tentative de réutilisation du même code ---")
system.authenticate_step1("charlie", "SuperSecure123!")
system.authenticate_step2("charlie", backup_code)

---

## 📊 Récapitulatif des concepts

### Hachage de mots de passe
- ✅ **bcrypt** avec cost ≥ 12
- ✅ **Salt unique** par utilisateur (automatique avec bcrypt)
- ✅ **Comparaison timing-safe** avec `hmac.compare_digest()` ou `bcrypt.checkpw()`
- ❌ **Jamais SHA-256 seul** pour les mots de passe

### Challenge-Response
- ✅ **HMAC** pour calculer les réponses
- ✅ **Challenges uniques** (nonces)
- ✅ **Timeout** sur les challenges
- ✅ **SCRAM** pour mutual authentication

### Multi-Factor Authentication
- ✅ **TOTP** (RFC 6238) avec fenêtre de tolérance
- ✅ **Codes de récupération** (usage unique)
- ✅ **QR codes** pour faciliter l'enrollment
- ✅ **Backup codes** conservés en lieu sûr

---

## 🎯 Points clés à retenir

1. **Ne jamais transmettre les mots de passe en clair**
2. **Utiliser bcrypt/scrypt/Argon2** pour le hachage
3. **Implémenter le 2FA** pour les applications critiques
4. **Prévoir des codes de récupération** pour le 2FA
5. **Utiliser des comparaisons timing-safe**
6. **Ajouter des timeouts** aux challenges/sessions